<a href="https://colab.research.google.com/github/FrantisekBrandejs/BAK-RQA/blob/main/BAK_RQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Využití RQA v kartografii** | bakalářská práce | František BRANDEJS, 2026

Prohlídnutí dat

In [4]:
import pandas as pd
data = pd.read_csv('drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv', sep = '\t', nrows = 10) #načtení jen prvních 10 řádků
print(data)
print(data.columns.to_list()) #výpis sloupců
!wc -l 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' #počet řádků
!cut -f555 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u #unikátní hodnoty ve specifickém sloupci
!cut -f19 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv' | sort -u #co za hodnoty ma eye movement type

   Recording timestamp  Computer timestamp       Sensor Participant name  \
0                    0       6578398691352          NaN           Standa   
1                82154       6578398773506          NaN           Standa   
2               107157       6578398798509  Eye Tracker           Standa   
3               110490       6578398801842  Eye Tracker           Standa   
4               113823       6578398805175  Eye Tracker           Standa   
5               117157       6578398808509  Eye Tracker           Standa   
6               120490       6578398811842  Eye Tracker           Standa   
7               123823       6578398815175  Eye Tracker           Standa   
8               127157       6578398818509  Eye Tracker           Standa   
9               130490       6578398821842  Eye Tracker           Standa   

  Zkusenosti  Student KGI Pohlavi    Vek Recording name               Event  \
0      Denne          NaN     Muz  31-40     Recording1      RecordingStart   
1    

Chunking zpracování a redukce dat

In [5]:
import pandas as pd
import re
import os
file_path = 'drive/MyDrive/BAK-RQA/DP-2024-Vitkova Data export.tsv'
out_path = 'drive/MyDrive/BAK-RQA/reduced_data.tsv'

non_fixation_columns = [
    'Participant name',
    'Recording timestamp',
    'Zkusenosti',
    'Eye movement type',
    'Gaze point X',
    'Gaze point Y'
    ]

#omezení sloupců s AOI na sloupec končící All a legendu (01 - 09)
aoi_columns_pattern = r'^AOI hit \[.* - (All|0[1-9])\]$'

#Kontrola správného výběru sloupců
header = pd.read_csv(file_path, sep='\t', nrows=0)
all_columns = header.columns.tolist()

# Filtrace pomocí aoi_columns_pattern
selected_aoi_columns = [col for col in all_columns if re.search(aoi_columns_pattern, col)]

print(f"Celkem nalezeno {len(selected_aoi_columns)} AOI sloupců.")
print("Výpis AOI sloupců pro kontrolu:")
print(selected_aoi_columns)

cols_to_keep = non_fixation_columns + selected_aoi_columns
print(f"Celkem vybráno {len(cols_to_keep)} sloupců.")

chunk_size = 500000  # Počet řádků v jedné várce
count = 0

reader = pd.read_csv(file_path, sep='\t', usecols=cols_to_keep, chunksize=chunk_size, low_memory=False)

for i, chunk in enumerate(reader):
    reduced_chunk = chunk[chunk['Eye movement type'] == 'Fixation'].copy() # Nechat jen řádky s fixacemi

    # Ukládání
    if i == 0:
        reduced_chunk.to_csv(out_path, sep='\t', index=False, mode='w') #zapsání prvního chunku i s hlavičkou
    else:
        reduced_chunk.to_csv(out_path, sep='\t', index=False, mode='a', header=False) #další chunky už bez hlavičky

    count += len(chunk)
    if (i + 1) % 5 == 0:  # Výpis každých 500 000 zpracovaných řádků
        print(f"Zpracováno {count:,} řádků...")

print(f"HOTOVO! Redukovaná data byla uložena do: {out_path}")
print(f"Redukovaná data: ")

Celkem nalezeno 428 AOI sloupců.
Výpis AOI sloupců pro kontrolu:
['AOI hit [R6bSV - 01]', 'AOI hit [R8dJV - 01]', 'AOI hit [R5aSV - 01]', 'AOI hit [R8bJV - 01]', 'AOI hit [R7cJV - 01]', 'AOI hit [R6cJV - 01]', 'AOI hit [R7cSV - 01]', 'AOI hit [R9dSV - 01]', 'AOI hit [R7aJV - 01]', 'AOI hit [R9gSV - 01]', 'AOI hit [R6aJV - 01]', 'AOI hit [R8aJV - 01]', 'AOI hit [R5bJV - 01]', 'AOI hit [R9eSV - 01]', 'AOI hit [R6bJV - 01]', 'AOI hit [R9bJV - 01]', 'AOI hit [R7bSV - 01]', 'AOI hit [R5bSV - 01]', 'AOI hit [R9cJV - 01]', 'AOI hit [R9cSV - 01]', 'AOI hit [R8bSV - 01]', 'AOI hit [R8fSV - 01]', 'AOI hit [R9bSV - 01]', 'AOI hit [R8eSV - 01]', 'AOI hit [R6cSV - 01]', 'AOI hit [R8cSV - 01]', 'AOI hit [R8dSV - 01]', 'AOI hit [R7eSV - 01]', 'AOI hit [R9aJV - 01]', 'AOI hit [R8aSV - 01]', 'AOI hit [R6dJV - 01]', 'AOI hit [R9gJV - 01]', 'AOI hit [R9fJV - 01]', 'AOI hit [R8eJV - 01]', 'AOI hit [R9eJV - 01]', 'AOI hit [R7dSV - 01]', 'AOI hit [R8fJV - 01]', 'AOI hit [R5aJV - 01]', 'AOI hit [R5cJV - 01]'

In [16]:
file_path = 'drive/MyDrive/BAK-RQA/reduced_data.tsv'

import pandas as pd
data = pd.read_csv(file_path, sep = '\t', nrows = 1) #načtení jen prvních 10 řádků
print(data)
print(data.columns.to_list()) #výpis sloupců
!wc -l 'drive/MyDrive/BAK-RQA/reduced_data.tsv' #počet řádků
!cut -f6 'drive/MyDrive/BAK-RQA/reduced_data.tsv' | sort -u # kontrola jestli redukovaná data jsou jen fixace

   Recording timestamp Participant name Zkusenosti  Gaze point X  \
0              1356314           Standa      Denne        1203.0   

   Gaze point Y Eye movement type  AOI hit [R6bSV - 01]  AOI hit [R8dJV - 01]  \
0         674.0          Fixation                   NaN                   NaN   

   AOI hit [R5aSV - 01]  AOI hit [R8bJV - 01]  ...  AOI hit [R7eJV - All]  \
0                   NaN                   NaN  ...                    NaN   

   AOI hit [R8cJV - All]  AOI hit [R7bJV - All]  AOI hit [R7dJV - All]  \
0                    NaN                    NaN                    NaN   

   AOI hit [R6dSV - All]  AOI hit [R9dJV - All]  AOI hit [R5cSV - All]  \
0                    NaN                    NaN                    NaN   

   AOI hit [R7aSV - All]  AOI hit [VZOR - All]  AOI hit [R9fSV - All]  
0                    NaN                   NaN                    NaN  

[1 rows x 434 columns]
['Recording timestamp', 'Participant name', 'Zkusenosti', 'Gaze point X', 'Gaze